In [23]:

%run closest_versus_minimal_tools.ipynb
import matplotlib.pyplot as plt

In [24]:
import itertools


def descent(jsonfilename):
    cases = from_json(jsonfilename)
    reached = []
    pathways = []
    i=1
    for case in cases:
        i+=1
        if i==42 or True:
            B, lc_cube, lc_LLL = case["B"], case["lincomb_cube"], [int(i) for i in case["lincomb_LLL"]]
            r, pathway = descent_single(B, lc_cube, lc_LLL)
            reached.append(r)
            pathways.append(pathway)
            # print(pathway[1:])
            # print("Reached?", r)
            if not r:
                plot_pathway_and_nearest_norms(pathway,r, case["lincomb_cube"], case["lincomb_LLL"], case["B"], neighborhood=4)
            # print("----------------------------")
        
    return reached, pathways


def descent_single(B, lc_cube, lc_LLL):
    pathway = []
    # index = get_index_to_remove(lc_cube, lc_LLL)
    # Append the first, real-valued point of the path. This is used later for plotting.
    pathway.append(lc_cube)
    LLL_norm = (matrix(lc_LLL) * matrix(B)).norm().n()

    # Find the point of the small cube with smallest norm and set it as a center
    min_norm, center = evaluate_norms_at_cube(cube_points(lc_cube), B)[0]
    print("💡: My first center is", center, "with norm", min_norm, ".")
     # If I reached the lcLLL within the first cube, end. TODO this is not a good way how to go around this, if there would be better one somewhere around, id never find it.
    if min_norm == LLL_norm:
        print("💡: Oh wow, I reached LLL in the very first step! The LLL is of norm", LLL_norm, "any my vertice of cube is of norm", min_norm)
        pathway.append(center)
        reached = True
        pathway.append(lc_LLL)
        return reached, pathway

        
    while True :
        pathway.append(center)
        big_cube = big_cube_points(center)
        print("💡: My new cube Im currently analyzing is ", big_cube)
        bigCubeMinimum_norm, bigCubeMinimum_lc = evaluate_norms_at_cube(big_cube, B)[0]
        print("💡: The smallest lc of it is", bigCubeMinimum_lc, "with norm ", bigCubeMinimum_norm)
        # If I reached the lcLLL, end with success status
        if bigCubeMinimum_norm == LLL_norm:
            pathway.append(bigCubeMinimum_lc)
            reached = True
            break

        # If my new minimum is smaller than LLL minimum
        elif bigCubeMinimum_norm < LLL_norm:
            pathway.append(bigCubeMinimum_lc)
            reached = True
            print("HEUREKA, IM BETTER THAN LLL!")
            break


        # If the new minimum is bigger than the old
        elif bigCubeMinimum_norm > min_norm:

            reached = False
            break

        # Else, continue with the descent
        min_norm, center = bigCubeMinimum_norm, bigCubeMinimum_lc
    return reached, pathway





In [25]:
# # This part of code is meant to generate the precise points the algorithm took during the descent. 


# import matplotlib.pyplot as plt

def different_components(pathway):
    diff_indices = set(range(len(pathway[0])))
    
    for i in range(len(pathway[0])):
        unique_values = {point[i] for point in pathway}
        if len(unique_values) == 1:
            diff_indices.remove(i)
            break

    return list(diff_indices)

# def plot_pathway(pathway, result):
#     '''
#     Plots the pathway of the descent algorithm in following manner:

#     red dot - lcCube
#     blue small dots - pathway of the descent
#     blue big dot - end of the descent, if not seen the descent ended successfully
#     orange big dot - lcLLL
#     '''
#     diff_indices = different_components(pathway)

#     x_pathway = [point[diff_indices[0]] for point in pathway]
#     y_pathway = [point[diff_indices[1]] for point in pathway]
    
#     # Create the plot
#     plt.scatter(x_pathway[-1],y_pathway[-1], color='orange',s=70)  # Highlight the last point lcLLL in green
#     plt.scatter(x_pathway[1:-1],y_pathway[1:-1], color='blue',s=10)
#     plt.scatter(x_pathway[0],y_pathway[0], color='red')  # Highlight the first point lcCUBE in red

#     # Add description next to the red point
#     description = f"({x_pathway[0]:.2f}, {y[0]:.2f})"
#     plt.text(x_pathway[0] + 0.1,y_pathway[0] - 0.1, description)

#     # Add descriptions to the blue points
#     for i in range(1, len(x) - 1):
#         description = f"({x_pathway[i]:.2f}, {y[i]:.2f})"
#         plt.text(x_pathway[i] + 0.1,y_pathway[i] - 0.1, description)

#     # Add gridlines corresponding to integers
#     plt.grid(True, which='both')

#     # Set axis ticks to integers only
#     plt.xticks(range(int(min(x)) - 1, int(max(x)) + 2))
#     plt.yticks(range(int(min(y)) - 1, int(max(y)) + 2))

#     # Show the plot
#     print("plot_pathway: reached LLL?", result)
    
#     plt.show()

In [36]:
# %run case_study_tools.ipynb

def plot_pathway_and_nearest_norms(pathway,lcCube, lcLLL, B, neighborhood, removecoordinateindex=None):
    if removecoordinateindex == None: 
        removecoordinateindex = get_index_to_remove(lcCube, lcLLL)
        if removecoordinateindex is not int:
            return ValueError("Please insert removecoordinateindex.")
    else:
        print("Note: LLL and cube do not share a coordinate!")
    pathway.append(lcLLL)
    fig = plt.figure(figsize=(5, 5), dpi=80)

    diff_indices = different_components(pathway)
    diff_indices = generate_nearby_integer_points(lcCube, removecoordinateindex, pathway)[0]

    # pathway points
    x_pathway = [point[diff_indices[0]] for point in pathway]
    y_pathway = [point[diff_indices[1]] for point in pathway]
    


    # scatter the pathway
    # plt.scatter(x_pathway[-1],y_pathway[-1], color='green',s=200, marker="*")  # highlight the last point lcLLL in green
    plt.scatter(x_pathway[1:-1],y_pathway[1:-1], color='orange',s=70) # pathway in orange
    plt.scatter(x_pathway[0],y_pathway[0], color='red', s=60)  # highlight the first point lcCUBE in red

    # # Add description next to the red point
    # description = f"({x_pathway[0]:.2f}, {y[0]:.2f})"
    # plt.text(x_pathway[0] + 0.1,y_pathway[0] - 0.1, description)

    # # Add descriptions to the blue points
    # for i in range(1, len(x) - 1):
    #     description = f"({x_pathway[i]:.2f}, {y[i]:.2f})"
    #     plt.text(x_pathway[i] + 0.1,y_pathway[i] - 0.1, description)

    # # Add gridlines corresponding to integers
    # plt.grid(True, which='both')

    # # Set axis ticks to integers only
    # plt.xticks(range(int(min(x)) - 1, int(max(x)) + 2))
    # plt.yticks(range(int(min(y)) - 1, int(max(y)) + 2))

    index =removecoordinateindex 
    nearby_integer_lcs = generate_nearby_integer_points(lcCube, removecoordinateindex, neighborhood)
    print("nilc",nearby_integer_lcs)

    nearby_removed, cube_removed, _ = remove_coordinate(removecoordinateindex, nearby_integer_lcs, lcCube, lcLLL)
    x_nearby = [point[0] for point in nearby_removed]
    y_nearby= [point[1] for point in nearby_removed]
    norms = [(vector(point)*matrix(B)).norm().n(digits=3) for point in nearby_integer_lcs]
    
    norm_min = min(norms)
    norm_max = max(norms)
    colors = [(norm - norm_min) / (norm_max - norm_min) for norm in norms]
    plt.scatter(x_nearby, y_nearby, c=colors, cmap='Blues', edgecolors='face', s=30)
    
    # Add norm as a description to each point with coords [x, y]
    for x, y, norm in zip(x_nearby, y_nearby, norms):
        plt.annotate(
            f"{norm}",
            xy=(x, y),
            xytext=(x + 0.1, y + 0.1),
            fontsize=10,
        )
    
    # Add gridlines corresponding to integers
    plt.grid(True, which='both')

    # Set axis ticks to integers only
    plt.xticks(range(int(min(x_nearby+ x_pathway)) - 1, int(max(x_nearby+ x_pathway)) + 2))
    plt.yticks(range(int(min(y_nearby+y_pathway)) - 1, int(max(y_nearby+y_pathway)) + 2))
    


    # Annotate LLL vs descend 
    LLLnorm = f'{"LLL "}{(vector(lcLLL)*matrix(B)).norm().n(digits=3)}{(x_pathway[-1],y_pathway[-1])}'

    # plt.annotate(LLLnorm, xy=(x_pathway[-1],y_pathway[-1]), xytext=(20,20), 
    #             textcoords='offset points', ha='center', va='bottom',
    #             bbox=dict(boxstyle='round,pad=0.2', fc='white', alpha=0.9),
    #             arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0.5', 
    #                         color='red'))
    plt.annotate('descended', xy=(x_pathway[-2],y_pathway[-2]), xytext=(-30,-30), 
                textcoords='offset points', ha='center', va='bottom',
                bbox=dict(boxstyle='round,pad=0.2', fc='orange', alpha=0.9),
                arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0.5', 
                            color='red'))

    # Show the plot
    # print(result)
    fig.show()


In [27]:
def get_index_to_remove(lcCube, lcLLL):
    '''
    Find the index of the identical coordinate shared by lcCube and lcLLL.

    INPUTS:
        lcCube (list): The lcCube point represented as a list of coordinates.
        lcLLL (list): The lcLLL point represented as a list of coordinates.
    '''
    if len(lcCube) != len(lcLLL):
        raise ValueError("Both input lists must have the same length")

    for index, (elem_lcCube, elem_lcLLL) in enumerate(zip(lcCube, lcLLL)):
        if elem_lcCube == elem_lcLLL:
            return index
    return ValueError("The points do not share identical coordinate.")


def generate_nearby_integer_points(lcCube, exception_index, dist):
    '''
    Generate nearby integer points of lcCube within a given distance, excluding a specific index.

    INPUTS:
        lcCube (list): The lcCube point represented as a list of coordinates.
        exception_index (int): The index of the coordinate to exclude from the calculation.
        dist (int): The maximum distance to generate nearby integer points.
    '''
    if dist<2:
        return ValueError("Neighborhood insufficiently large.")

    n = len(lcCube)
    nearby_points = []
    
    # Calculate the range of possible integer coordinates for each dimension, except at index ind
    ranges = [range(int(round(coord) - dist + 1), int(round(coord) + dist)) if i != exception_index else [lcCube[exception_index]] for i, coord in enumerate(lcCube)]

    # Generate all possible combinations of integer coordinates within the ranges
    coordinate_combinations = list(itertools.product(*ranges))

    for point in coordinate_combinations:
        total_diff = sum(abs(lcCube[i] - point[i]) for i in range(n))
        if total_diff < dist:
            nearby_points.append(list(point))
    
    return nearby_points


def remove_coordinate(index, points, lcCube, lcLLL):
    '''
    Remove a coordinate from a list of points, lcCube, and lcLLL at the specified index.
    '''
    filtered_points = [remove_coordinate_single(index, point) for point in points]
    lcCubeRemoved = remove_coordinate_single(index, lcCube)
    lcLLLRemoved = remove_coordinate_single(index, lcLLL)
    return filtered_points, lcCubeRemoved, lcLLLRemoved


def remove_coordinate_single(index, point):
    '''
    Remove a coordinate from a single point at the specified index.
    INPUTS:
        index (int): The index of the coordinate to be removed.
        point (list): The point represented as a list of coordinates.
    '''
    point_removed = point[:]
    point_removed.pop(index)
    return point_removed

In [31]:
from math import floor, ceil

def small_cube(floats):
    n = len(floats)
    seen_combinations = set()  
    for i in range(2**n):
        combination = []
        for j in range(n):
            if i & (1 << j):
                combination.append(ceil(floats[j]))
            else:
                combination.append(floor(floats[j]))
        seen_combinations.add(tuple(combination))  # Convert list to tuple for hashability
    return [list(comb) for comb in seen_combinations]

# Example
# floats = [1.4, 5.6]
# unique_roundings = generate_roundings_no_duplicates(floats)
# print(unique_roundings)



# # # # Example usage
# # result = generate_roundings_no_duplicates([3.0, -10.320968627929688, 12.099395751953125])
# print(result)




[[1, 6], [2, 5], [2, 6], [1, 5]]
[[3, -11, 12], [3, -10, 12], [3, -11, 13], [3, -10, 13]]


In [37]:
def generate_plus_minus_ones(numbers):
    """
    Generates all combinations of the given numbers with +-1 applied to each entry.

    Args:
    - numbers: List[int] - list of integers.

    Returns:
    - List[List[int]] - list of lists, each representing one combination.
    """
    result = []
    for i, num in enumerate(numbers):
        # create copies of the original list with +1 and -1 applied to the current number
        plus_one = numbers[:i] + [num + 1] + numbers[i+1:]
        minus_one = numbers[:i] + [num - 1] + numbers[i+1:]
        result.extend([plus_one, minus_one])
    return result

# example usage
print(generate_plus_minus_ones([1, 5]))


[[2, 5], [0, 5], [1, 6], [1, 4]]
